In [1]:
!mc cp -r s3/$VAULT_TOP_DIR/Accords/chroma_db .

]11;?\mc: Configuration written to `/home/onyxia/.mc/config.json`. Please update your access credentials.
mc: Successfully created `/home/onyxia/.mc/share`.
mc: Initialized share uploads `/home/onyxia/.mc/share/uploads.json` file.
mc: Initialized share downloads `/home/onyxia/.mc/share/downloads.json` file.
...ma.sqlite3: 221.59 MiB / 221.59 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 182.67 MiB/s 1s

In [2]:
!uv pip install -q langchain==0.2.16 matplotlib langchain-community==0.2.16 langchain-huggingface==0.0.3 langchain-text-splitters==0.2.4 chromadb==0.5.3 langchain-chroma==0.1.3 pandas ipywidgets

In [3]:
!uv pip install -q pyarrow

In [16]:
!uv pip install -q unidecode

In [4]:
import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings, OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.schema import Document, Generation, LLMResult
from langchain.llms import Ollama, BaseLLM
from langchain.chains import StuffDocumentsChain, RetrievalQA, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from pathlib import Path
import json
import requests
import re

In [17]:
import unidecode


In [5]:
from glob import glob

In [6]:
class LocalOllamaLLM(BaseLLM):
    api_url : str
    def _generate(self, prompt, stop):
        response = requests.post(f"{self.api_url}/api/generate", json={"model": "llama3.1", "prompt": str(prompt) })
        response.raise_for_status()
        response_text=''.join([json.loads(line)['response'] for line in response.text.splitlines()])
        generations=[]
        generations.append([Generation(text=response_text)])
        return LLMResult(generations=generations)


    def _llm_type(self):
        return "local"  # Or whatever type is appropriate for your local setup

llm = LocalOllamaLLM(api_url="http://127.0.0.1:11434")

embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

system_prompt = (
    " Répondez à la question posée "
    " Utilisez le contexte (sélection des meilleurs paragraphes liés à la question) donné pour répondre à la question "
    " Répondez en bullet point comme ceci en copie-collant (sans reformulation) les phrases exactes issues du contexte qui t'ont permis de répondre. : "
    " ### Règles : "
    " * (règle 1) "
    " * (règle 2) "
    " Si la réponse ne se trouve pas dans le contexte, répondez par 'Null'"
    

    " Contexte : {context}  "
)




prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

""" system_prompt_bis = (
    " Répondez à la question posée "
    " Utilisez le contexte (sélection des meilleurs paragraphes liés à la question) donné pour répondre à la question "
    "Répondez comme ceci :"
    "1. Un objet JSON avec les champs suivants :"
     " - contingent_annuel : nombre entier en jour par année (ou null s’il est non défini ou non déductible),"
     " - regle_majoration : true ou false selon s’il existe des règles de majoration des heures supplémentaires,"
     "- repos_compensateur_possible : true ou false selon que le paiement des heures supplémentaires peut être remplacé par un repos compensateur."
    "2. Ensuite, réécrire les phrases extraites du tete qui ont permi de répondre à ces questions"
    " Commencez impérativement ta réponse par la section JSON, puis ajoutez l’explication."
    
    #" Si la réponse ne se trouve pas dans le contexte, répondez par 'Non'"
    " Contexte : {context}  
)
"""
"""
system_prompt_bis = (
    "Tu es un assistant chargé d'extraire des informations à partir d'un contexte fourni. "
    "Ta tâche est de répondre à des questions en produisant un objet JSON unique avec des champs standardisés. "
    "Voici comment structurer ta réponse :\n\n"
    Ta réponse doit toujours être structurée sous forme de JSON avec les champs suivants :\n"
    "- contingent_annuel : un nombre entier représentant le contingent annuel d'heures supplémentaires (en heures). "
    "Voici le contexte : {context}"
)
"""

system_prompt_bis = (
        "Tu es un assistant intelligent chargé d'extraire des informations précises à partir d'un contexte donné. "
        "Ta tâche est de répondre à une question spécifique en produisant uniquement un objet JSON nommé `variable`, avec des champs standardisés.\n\n"
        "Tu dois retourner un JSON structuré sous la forme suivante :\n"
        "variable = {{\n"
            '  "contingent_annuel": <valeur entière en heures ou null si non précisé>\n'
        "}}\n\n"
        "Important :\n"
        "- La réponse doit être uniquement le JSON"
        "- Si le contingent annuel n’est pas clairement indiqué dans le contexte, la valeur doit être `null` (sans guillemets).\n"
        "- Ne fais aucun commentaire ni explication autour du JSON.\n\n"
        "Voici le contexte : {context}"
)




system_prompt_bis = (
    "Tu es un assistant intelligent chargé d'extraire une information dans un contexte donné. "
    "Ta tâche est de répondre à la question suivante : « À combien est fixé le contingent d’heures supplémentaires ? »\n\n"
    "Tu dois répondre uniquement par un JSON au format strict suivant :\n"
    "{{\n"
    '  "contingent_annuel": <valeur numérique entière ou null>\n'
    "}}\n\n"
    "- Si l’information n’est pas présente, mets la valeur `null` (sans guillemets).\n"
    "- Ne produis aucun commentaire, explication ou texte hors du JSON.\n\n"
    "Contexte : {context}"
)



system_prompt_bis = (
    "Tu es un assistant intelligent chargé d'extraire une information dans un contexte donné. "
    "Ta tâche est de répondre à la question suivante : « À combien est fixé le contingent d’heures supplémentaires ? »\n\n"
    "Tu dois répondre uniquement par un JSON au format strict suivant :\n"
    "{{\n"
    '  "contingent_annuel": <valeur numérique entière ou null>\n'
    "}}\n\n"
    "- Si l’information n’est pas présente, indique la valeur `null` (sans guillemets).\n"
    "- Ne produis aucun commentaire, explication ou texte hors du JSON.\n\n"
    "Attention : le **contingent d’heures supplémentaires** (également appelé **contingent annuel d’heures supplémentaires** ou parfois **quota d’heures supplémentaires**) "
    "désigne le **nombre maximal d’heures supplémentaires qu’un salarié est autorisé à effectuer sur une année**.\n"
    "Il **ne s’agit pas** du seuil hebdomadaire à partir duquel une heure devient supplémentaire.\n"
    "Par exemple, une phrase comme : « Constituent des heures supplémentaires celles effectuées au-delà de 37 heures » ne permet pas d’identifier le contingent annuel et doit conduire à renvoyer `null`.\n"
    " Mets `null` si l'information ne concerne **pas strictement** le contingent annuel d’heures supplémentaires.\n\n"
    "Contexte : {context}"
)


In [7]:
system_prompt_bis = (
    "Tu es un assistant intelligent chargé d'extraire une information dans un texte.\n"
    "Réponds uniquement par un JSON au format suivant :\n"
    "{{\n"
    '  "contingent_annuel": <valeur numérique entière ou null>\n'
    "}}\n\n"
    "Le contingent annuel d’heures supplémentaires est le nombre total d’heures supplémentaires autorisées pour un salarié sur une année.\n"
    "Ignore toute autre information comme le seuil de déclenchemnt des heures supplmémentaires ou la durée annuelle de travail de référence.\n"
    "Si l’information n’est pas clairement donnée, réponds avec null.\n\n"
    "Contexte : {context}"
)

prompt_bis = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_bis),
        ("human", "{input}"),
    ]
)
question_answer_chain_bis= create_stuff_documents_chain(llm, prompt_bis)

In [7]:
df = pd.read_parquet("data/echantillon_1000_hs_2024_TOC.parquet")
df = df.rename(columns={"numdossier_new":"numdossier"})
df= df.set_index("numdossier")

In [9]:
df

,accorddocx,txt_trim_list,extracted_summary
numdossier,,,
T00724060165,\n\nACCORD COLLECTIF D’ENTREPRISE\n \nEntre \n...,"[, , ACCORD COLLECTIF D’ENTREPRISE, , Entre ,...","[ACCORD COLLECTIF D’ENTREPRISE, PREAMBULE,  ..."
T03324062903,\t\t\n\t\nACCORD D’ENTREPRISE RELATIF A L’ORGA...,"[\t\t, \t, ACCORD D’ENTREPRISE RELATIF A L’ORG...",[ACCORD D’ENTREPRISE RELATIF A L’ORGANISATION ...
T07424060533,AVENANT N°2 DE L’ACCORD COLLECTIF D’ENTREPRISE...,[AVENANT N°2 DE L’ACCORD COLLECTIF D’ENTREPRIS...,[AVENANT N°2 DE L’ACCORD COLLECTIF D’ENTREPRIS...
T03224060321,\n\nACCORD D’ENTREPRISE SUR LES HEURES SUPPLEM...,"[, , ACCORD D’ENTREPRISE SUR LES HEURES SUPPLE...",[ACCORD D’ENTREPRISE SUR LES HEURES SUPPLEMENT...
T06724061948,\naccord d’entreprise \nrelatif au contingent ...,"[, accord d’entreprise , relatif au contingent...","[accord d’entreprise , relatif au contingent d..."
...,...,...,...
T07424060477,ACCORD COLLECTIF INSTITUANT UN AMENAGEMENT DU ...,[ACCORD COLLECTIF INSTITUANT UN AMENAGEMENT DU...,[ACCORD COLLECTIF INSTITUANT UN AMENAGEMENT DU...
T06224062102,\t\nACCORD D’ENTREPRISE SUR L’AMENAGEMENT DU T...,"[\t, ACCORD D’ENTREPRISE SUR L’AMENAGEMENT DU ...",[ACCORD D’ENTREPRISE SUR L’AMENAGEMENT DU TEMP...
T05124061033,\n\n\nACCORD SUR LES MODALITES D’ORGANISATION ...,"[, , , ACCORD SUR LES MODALITES D’ORGANISATION...",[ACCORD SUR LES MODALITES D’ORGANISATION DU TE...


In [10]:
vector_store = Chroma(embedding_function=embedder,persist_directory="./chroma_db")

In [11]:
Q1= "À combien est fixé le contingent d’heures supplémentaires ? "

In [12]:
def search_and_invoke_llm(vector_store, numdossier, query, question_answer_chain, k=5):
    if k == 0:
        print(f"Échec de la recherche pour le dossier {numdossier}")
        return None

    try:
        retriever = vector_store.as_retriever(
            search_kwargs={
                "k": k,
                "filter": {"numdossier": str(numdossier)}
            }
        )
        
        retrieved_docs = retriever.invoke(query)
        chain = create_retrieval_chain(retriever, question_answer_chain)
        result = chain.invoke({"input": query})

        return {
            "answer": result.get("answer", ""),
            "context": [doc.page_content for doc in retrieved_docs]
        }

    except Exception as e:
        print(f"Erreur avec k={k}, tentative avec k={k-1} pour dossier {numdossier} : {e}")
        return search_and_invoke_llm(vector_store, numdossier, query, question_answer_chain, k=k-1)
        
def save_results(df, path, question_answer_chain): 
    Path(path).mkdir(exist_ok=True)
    already_done = {el.split("/")[1].split(".")[0] for el in glob(f"{path}/*.json")}

    for index, row in df.iterrows():
        if str(index) not in already_done:
            numdossier = str(index)

            result_json = {
                "num_dossier": numdossier,
                "responses": {},
                "context": {}
            }

            result = search_and_invoke_llm(vector_store, numdossier, Q1, question_answer_chain, k=2)

            if result:
                result_json["responses"][Q1] = result.get("answer", "")
                result_json["context"][Q1] = result.get("context", [])
            else:
                result_json["responses"][Q1] = None
                result_json["context"][Q1] = []

            with open(f"{path}/{numdossier}.json", "w", encoding="utf-8") as f:
                json.dump(result_json, f, indent=2, ensure_ascii=False)


In [13]:
!sudo apt update && sudo apt install --yes lshw

Get:1 https://apt.postgresql.org/pub/repos/apt jammy-pgdg InRelease [129 kB]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:3 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]                
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://apt.postgresql.org/pub/repos/apt jammy-pgdg/main amd64 Packages [614 kB]3m
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,798 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:8 https://ppa.launchpadcontent.net/git-core/ppa/ubuntu jammy InRelease [24.6 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,476 kB]
Get:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]     
Get:12 http://archive.ubuntu.com/ubuntu jammy/

In [14]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [16]:
save_results(df,"results_llma_3_1_v1_2024_better_prompt",question_answer_chain_bis)

In [17]:
json_folder = "results_llma_3_1_v1_2024_better_prompt"
rows = []

# regex
contingent_pattern = re.compile(r'"contingent_annuel"\s*:\s*(\d+|null)')

for filepath in glob(f"{json_folder}/*.json"):
    with open(filepath, "r", encoding="utf-8") as f:
        data = json.load(f)
        num_dossier = data.get("num_dossier")

        # Récupération de la réponse brute
        response_text = data["responses"].get(
            "À combien est fixé le contingent d’heures supplémentaires ? "
        )

        contingent = None  # valeur par défaut

        if response_text:
            response_text = response_text.strip("` \n")
            match = contingent_pattern.search(response_text)
            if match:
                value = match.group(1)
                contingent = int(value) if value.isdigit() else None

        rows.append({
            "num_dossier": num_dossier,
            "contingent_annuel": contingent
        })

# Création et export CSV
df = pd.DataFrame(rows)
df.to_csv("contingents_annuels_betterprompt.csv", index=False)


In [18]:
results_llama3_1 = pd.read_csv("contingents_annuels_betterprompt.csv")
true_results= pd.read_csv("echantillon_eval_hs_2024_contingent_annote.csv")
results_llama3_1= results_llama3_1.rename(columns={"contingent_annuel":"contingent"})

In [19]:
true_results = true_results.rename(columns={"numdossier_new":"num_dossier"})

In [20]:
true_results

,num_dossier,contingent
0,T00724060165,NaN
1,T03324062903,400.0
2,T07424060533,NaN
3,T03224060321,NaN
4,T06724061948,360.0
...,...,...
95,T09224067019,NaN
96,T09224068199,NaN
97,T04524060557,220.0
98,T59L24063310,NaN


In [21]:
comparaison_df = pd.merge(
    results_llama3_1,
    true_results,
    on="num_dossier",
    how="inner",
    suffixes=('_llm', '_annot')  
)

In [22]:
comparaison_df

,num_dossier,contingent_llm,contingent_annot
0,T04524061314,NaN,NaN
1,T06924064719,220.0,220.0
2,T09224067998,330.0,330.0
3,T07524069698,220.0,220.0
4,T06624060247,320.0,320.0
...,...,...,...
95,T04524061142,450.0,450.0
96,T05724061388,300.0,300.0
97,T05624061314,265.0,265.0
98,T04524060557,220.0,220.0


In [23]:
comparaison_df["contingent_llm"] = comparaison_df["contingent_llm"].fillna("")
comparaison_df["contingent_annot"] = comparaison_df["contingent_annot"].fillna("")


In [26]:
comparaison_df[comparaison_df["contingent_llm"] != comparaison_df["contingent_annot"]]

,num_dossier,contingent_llm,contingent_annot
5,T09424060886,,250.0
13,T03324061515,400.0,500.0
36,T04524061125,220.0,
42,T06824060827,,270.0
49,T07524064067,1607.0,
83,T07524067789,,100.0


In [27]:
comparaison_df[comparaison_df["contingent_llm"] == comparaison_df["contingent_annot"]]

,num_dossier,contingent_llm,contingent_annot
0,T04524061314,,
1,T06924064719,220.0,220.0
2,T09224067998,330.0,330.0
3,T07524069698,220.0,220.0
4,T06624060247,320.0,320.0
...,...,...,...
95,T04524061142,450.0,450.0
96,T05724061388,300.0,300.0
97,T05624061314,265.0,265.0
98,T04524060557,220.0,220.0


In [12]:
df

,accorddocx,txt_trim_list,extracted_summary
numdossier,,,
T00724060165,\n\nACCORD COLLECTIF D’ENTREPRISE\n \nEntre \n...,"[, , ACCORD COLLECTIF D’ENTREPRISE, , Entre ,...","[ACCORD COLLECTIF D’ENTREPRISE, PREAMBULE,  ..."
T03324062903,\t\t\n\t\nACCORD D’ENTREPRISE RELATIF A L’ORGA...,"[\t\t, \t, ACCORD D’ENTREPRISE RELATIF A L’ORG...",[ACCORD D’ENTREPRISE RELATIF A L’ORGANISATION ...
T07424060533,AVENANT N°2 DE L’ACCORD COLLECTIF D’ENTREPRISE...,[AVENANT N°2 DE L’ACCORD COLLECTIF D’ENTREPRIS...,[AVENANT N°2 DE L’ACCORD COLLECTIF D’ENTREPRIS...
T03224060321,\n\nACCORD D’ENTREPRISE SUR LES HEURES SUPPLEM...,"[, , ACCORD D’ENTREPRISE SUR LES HEURES SUPPLE...",[ACCORD D’ENTREPRISE SUR LES HEURES SUPPLEMENT...
T06724061948,\naccord d’entreprise \nrelatif au contingent ...,"[, accord d’entreprise , relatif au contingent...","[accord d’entreprise , relatif au contingent d..."
...,...,...,...
T07424060477,ACCORD COLLECTIF INSTITUANT UN AMENAGEMENT DU ...,[ACCORD COLLECTIF INSTITUANT UN AMENAGEMENT DU...,[ACCORD COLLECTIF INSTITUANT UN AMENAGEMENT DU...
T06224062102,\t\nACCORD D’ENTREPRISE SUR L’AMENAGEMENT DU T...,"[\t, ACCORD D’ENTREPRISE SUR L’AMENAGEMENT DU ...",[ACCORD D’ENTREPRISE SUR L’AMENAGEMENT DU TEMP...
T05124061033,\n\n\nACCORD SUR LES MODALITES D’ORGANISATION ...,"[, , , ACCORD SUR LES MODALITES D’ORGANISATION...",[ACCORD SUR LES MODALITES D’ORGANISATION DU TE...


In [9]:
def normalize(text):
    return unidecode.unidecode(text.lower().strip())

def split_text_by_sentences(text, flagged_sentences):
    """
    Découpe le texte en segments basés sur les titres du sommaire, après normalisation.
    """
    split_texts = []
    positions = []

    normalized_text = normalize(text)

    # On garde un mapping (titre original, position) pour préserver les titres initiaux
    for sentence in flagged_sentences:
        norm_sentence = normalize(sentence)
        pos = normalized_text.find(norm_sentence)
        if pos != -1:
            # On retrouve la position réelle dans le texte original
            real_pos = text.lower().find(sentence.lower())
            if real_pos != -1:
                positions.append(real_pos)

    # Si aucune position trouvée, retourner le texte complet
    if not positions:
        return [text]

    positions = sorted(set(positions))
    positions.insert(0, 0)
    positions.append(len(text))

    for i in range(len(positions) - 1):
        start = positions[i]
        end = positions[i + 1]
        split_texts.append(text[start:end].strip())

    return split_texts

In [10]:
def split_text_with_titles(text, summary_titles):
    chunks = split_text_by_sentences(text, summary_titles)
    result = {}
    for title in summary_titles:
        for chunk in chunks:
            if normalize(title) in normalize(chunk[:len(title)+30]):
                result[title] = chunk.strip()
                break
    return result


In [18]:
df["section_dict"] = df.apply(
    lambda row: split_text_with_titles(row["accorddocx"], row["extracted_summary"]),
    axis=1
)

In [20]:
df["section_dict"].iloc[0]                                                 

{'ACCORD COLLECTIF D’ENTREPRISE': 'ACCORD COLLECTIF D’ENTREPRISE\n\xa0\nEntre\xa0\n\xa0\nL’Association Mission Locale Nord Ardèche, dont le siège est situé 6 allée Nicolas Copernic- Tour H représenté par M.                   en sa qualité de Président.\n\xa0\nD’une part,\xa0\xa0\n\xa0\nEt\n\xa0\nLes délégués titulaires du CSE, représentant la majorité des suffrages exprimés lors des dernières élections professionnelles en date du 10/07/2023.\n\xa0\nD’autre part,',
 'PREAMBULE': 'PREAMBULE\n\xa0\nLes parties ont convenu de négocier un nouvel accord collectif d’entreprise suite à la dénonciation en date 06/12/2022 de l’accord collectif d’entreprise du 23/12/1999.\n\xa0\nLe présent accord prend en compte l’organisation actuelle en répondant aux besoins d’adaptation de la Mission Locale et à ses nécessités de fonctionnement.\n\xa0\nLe présent accord comporte notamment des dispositions concernant\xa0:',
 '\uf0b7\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0L’aménagement du temps de travail\xa0;': '\u

In [28]:
df["section_dict"][0].keys()

/tmp/ipykernel_1238/2344104888.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df["section_dict"][0].keys()


dict_keys(['ACCORD COLLECTIF D’ENTREPRISE', 'PREAMBULE', '\uf0b7\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0L’aménagement du temps de travail\xa0;', 'TITRE I – CHAMP D’APPLICATION', 'TITRE II – DUREE DU TRAVAIL, DUREES MAXIMALES, REPOS HEBDOMADAIRE', '1. Durée conventionnelle de travail', '2. Durées maximales de travail', '3. Repos hebdomadaire', 'TITRE III – AMENAGEMENT DU TEMPS DE TRAVAIL', '1. Aménagement du temps de travail sur une période annuelle', '2. Période annuelle de référence', '3. Salariés à temps complet', '4. Salariés à temps partiel', '5. Planification de l’aménagement du temps de travail', '6. Horaires de travail - modification de l’horaire', '7. Lissage de la rémunération – absences – arrivées et départs en cours de période de référence', '8. Heures supplémentaires', '9. Heures complémentaires', '10. Travail en soirée', '11. Travail le samedi', '12. Suivi des horaires et temps de travail', '13. Temps de travail des cadres', '13.1. Cadres dirigeants', '13.2. Cadres administrat

In [29]:
df.iterrows()

<generator object DataFrame.iterrows at 0x7f824395e460>